# Introduction
This notebook demonstrates the creation of a set of tools and an agent designed to assist in handling insurance claims. We'll walk through setting up necessary tools that analyze text data to detect damages, calculate costs, and check online price ranges. These tools will then be integrated into an agent capable of processing insurance claims efficiently.

## Setup - Imports & Definition of Tools
In this section, we import necessary libraries and define custom tools. Each tool serves a specific purpose:
- `damage_detector`: Identifies damaged items and their condition from text.
- `calculator`: Estimates repair costs based on descriptions.
- `online_checker`: Verifies if the pricing of items is reasonable based on online data.


In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.agents import AgentExecutor
from langchain.agents import tool
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import (
    OpenAIToolsAgentOutputParser,
)
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pyprojroot import here


In [2]:
@tool
def damage_detector(text: str) -> str:
    """
    Identifies and lists damaged items from a text description, 
    including the extent of damage when available.
    """

    template = (
     "text: {text} \n\n Instruction: "
     "Create a list of all damaged objects. Indicate, if applicable, "
     "the extent of the damage for each item."
    )
    
    prompt = PromptTemplate(
        input_variables=["text"],
        template=template,
    )

    summary_chain = prompt | ChatOpenAI(
        openai_api_key=api_key,
        temperature=0,
        model_name="gpt-4",
    )

    summary = summary_chain.invoke({"text": text})

    return summary.content

In [3]:
@tool
def calculator(text: str) -> str:
    """
    Estimates the total costs for damaged objects to interpret 
    and compute values based on textual descriptions.
    """

    template = (
        "text: {text} \n\n Instruction: Calculate the total costs"
        "of the damaged objects"
    )
    
    prompt = PromptTemplate(
        input_variables=["text"],
        template=template,
    )

    calculator_chain = prompt | ChatOpenAI(
        openai_api_key=api_key,
        temperature=0,
        model_name="gpt-4",
    )

    cost = calculator_chain.invoke({"text": text})

    return cost.content

In [4]:
@tool
def online_checker(text: str) -> str:
    """
    Checks on the web weather the price for an item is supposed to be in a reasonable range
    """

    search_engine = DuckDuckGoSearchRun()

    result = search_engine.run(text)

    return result

## Loading API Key & Data
We utilize the `dotenv` package to securely load environment variables, such as API keys, to ensure our application's security. The data for this notebook is loaded from a `damage_report.pdf`, which contains descriptions of damaged items that our tools will process.


In [5]:
load_dotenv(find_dotenv())
api_key = os.getenv("OPENAI_API_KEY")

In [6]:
data = PyPDFLoader(str(here("./damage_report.pdf"))).load()
print(data)

[Document(page_content='Property Damage Report - Storm Damage\nDear Sir/Madam,\nI am writing to report damage incurred to my property during the storm on February 22, 2024. As a\nholder of a home contents insurance policy, I am seeking prompt attention to this matter.\nA severe storm caused significant damage to my residence in the southern part of Cologne,\nspecifically at Mainzer Straße 45, 50678 Köln. The damage includes broken windows allowing water\nto enter and damage various household items including furniture and electronic devices.\nThe following items were affected: \n- Oak living room table, water damage, estimated current value: EUR 200\n- 55-inch Samsung television, water damage, estimated replacement value: EUR 800\n- Lenovo laptop, water damage, estimated replacement value: EUR 1200\n- Various books and decorative items, estimated current value: EUR 150\nI have taken all necessary measures to prevent further damage and ensure safety. I also have\nphotos of the damaged it

## Definition of Tools & Agent
Here, we define the agent and bind it with the previously defined tools. The agent uses these tools to interpret and process the information from the insurance claims:
- The `damage_detector` tool extracts and lists damaged items.
- The `calculator` computes the cost estimates.
- The `online_checker` validates the cost estimates against current market prices.


In [7]:
tools = [damage_detector, calculator, online_checker]
llm = ChatOpenAI(model_name="gpt-4", openai_api_key=api_key, temperature=0)

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "I am a dedicated assistant for insurance claims handling. "
            "I help process relevant information and provide cost overviews for claims. "
            "However, I do not have access to current events. "
            "It is essential for the claims handler to verify the plausibility of the "
            "provided cost estimations to ensure they are realistic."
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [9]:
llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

## Agent Execution
In this final step, we execute our agent with the bound tools to process a sample insurance claim. The `AgentExecutor` class manages the orchestration between the agent and tools, handling inputs and formatting outputs. The result is a comprehensive analysis of the claim, ready for review and further processing.


In [10]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke(
    {"input": data[0].page_content}, return_only_outputs=True
)



> Entering new AgentExecutor chain...

Invoking: `damage_detector` with `{'text': 'Oak living room table, water damage, estimated current value: EUR 200\n55-inch Samsung television, water damage, estimated replacement value: EUR 800\nLenovo laptop, water damage, estimated replacement value: EUR 1200\nVarious books and decorative items, estimated current value: EUR 150'}`


1. Oak living room table - Extent of damage: Water damage
2. 55-inch Samsung television - Extent of damage: Water damage
3. Lenovo laptop - Extent of damage: Water damage
4. Various books and decorative items - Extent of damage: Not specified
Invoking: `calculator` with `{'text': 'Oak living room table, water damage, estimated current value: EUR 200\n55-inch Samsung television, water damage, estimated replacement value: EUR 800\nLenovo laptop, water damage, estimated replacement value: EUR 1200\nVarious books and decorative items, estimated current value: EUR 150'}`


The total cost of the damaged objects is EUR 23

Upon executing our agent, the output produced is an analyzed summary of the first page of the `damage_report.pdf`. This output reflects the combined capabilities of our tools:
- **Damage Identification**: Items and their damages listed as detected by `damage_detector`.
- **Cost Calculation**: Estimated repair costs provided by `calculator`.
- **Price Verification**: Online market price comparisons by `online_checker`.

This result showcases the practical application of our tools in automating aspects of insurance claim processing, reducing manual effort, and improving accuracy and speed of response.

